In [ ]:
# A2.1

# https://www.geogebra.org/calculator
# Q1:
# la ligne -: y+2x-2    = 0
# la ligne +: y+2x-10 = 0
# + 2 x2 + 1 x1 - 6 = 0

# Q2:
# +

# Q3:
# (0, 2)
# (1, 0)
# (3, 4)
# (4, 2)

# Q4:
# x1 = 2
# 5x1−x2−10=0

# Q5:
# (-1, 0)

# Q6:
# sqrt(3)*x2
# sqrt(6)*x1x2
# sqrt(3)*x1
# 1
# x2^3
# x1^3

# Q7:
# Some of
# with an appropri
# This set of examples is separable in the new feature space

# Q8:
# sqrt(3)x1^2x2
# x2^3
# x1^3
# sqrt(3)x1x2^2

# Q9: Yes

# Q10: g(x) = -x + 3 && best lin discriminant has acc of 80% on data

# Q11: - 1 x^2 - 1 x + 14.666

# Q12: The discriminant correspond to 2 decisions boundaries
# The discriminant is a convex quadratic function

In [8]:
# A2.2
import pandas as pd
import numpy as np
train_df = pd.read_csv("HeartTrain.csv", index_col=0)
test_df = pd.read_csv("HeartTest.csv", index_col=0)
def accuracy(pred, true):
    well = 0
    total = 0
    for i in range(len(pred)):
        if pred[i] == true[i]: 
            well += 1
        total += 1
    return well / total

# Q1:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
from sklearn.svm import SVC
from sklearn.model_selection import KFold

def KF_validation(X, Y, kernel, degree): 
    KF = KFold(n_splits=10, shuffle=False)
    sum_acc = 0
    for train_index, test_index in KF.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        model = SVC(kernel=kernel, degree=degree).fit(X_train, Y_train)
        sum_acc += accuracy(model.predict(X_test), Y_test)
    return sum_acc/10

cv_acc = KF_validation(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'linear', 3)
print("Q1: ", cv_acc)

# Q2:
maxi = 0.0
best = 0.0
for degree in range(20):
    cv_acc = KF_validation(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'poly', degree)
    if cv_acc > maxi:
        best = degree
        maxi = cv_acc
cv_acc = KF_validation(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'poly', 16)
print("Q2: ", cv_acc)

# Q3: 
from sklearn import preprocessing

# Remove not relevant feature
# BadList is list of index to columns with not enough correlation with the output variable
# badList = []
# corrLabels = train_df.corr()["labels"]
# for i in range(len(corrLabels)):
#     if (-0.3 < corrLabels[i] < 0.3):
#         badList.append(i+1)
# for i in badList:
#     train_df = train_df.drop("F_{}".format(i), axis=1)

# Warnings talked about scaler. I tested StandardScaler(), MinMaxScaler() & RobustScaler()
# StandardScaler() was the best one for the three kernels
columns = train_df.columns.to_numpy()
index = train_df.index.to_numpy()
rescale = preprocessing.StandardScaler()
train_df = pd.DataFrame(rescale.fit_transform(train_df), index=index, columns=columns)
train_df.loc[train_df["labels"]>0, 'labels']=1
train_df.loc[train_df["labels"]<-0.5, 'labels']=0

def KF_validation_Q3(X, Y, kernel, degree, C, gamma, coef0, tol, verbose, max_iter, decision_function_shape, break_ties): 
    KF = KFold(n_splits=10, shuffle=False)
    sum_acc = 0
    for train_index, test_index in KF.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        model = SVC(kernel=kernel, degree=degree, C=C, gamma=gamma, coef0=coef0, tol=tol, verbose=verbose, max_iter=max_iter, decision_function_shape=decision_function_shape, break_ties=break_ties).fit(X_train, Y_train)
        sum_acc += accuracy(model.predict(X_test), Y_test)
    return sum_acc/10

# maxi = 0.0
# best = {}
# for C in [1.0, 5.0, 20.0, 50.0, 100.0]:
#     for gamma in ['scale', 'auto']:
#         for coef0 in [-10.0, 0.0, 5.0, 8.0, 10.0, 30.0]:
#             for tol in [0.001, 0.01, 0.1, 1.0, 30.0]:
#                 for verbose in [False, True]:
#                     for max_iter in [-1]:
#                         for decision_function_shape in ['ovr', 'ovo']:
#                             for break_ties in [False, True]:
#                                 for degree in [1, 3, 5, 9, 16, 20]:
#                                     if not(break_ties and decision_function_shape=='ovo'):
#                                         cv_acc = KF_validation_Q3(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'poly', degree, C, gamma, coef0, tol, verbose, max_iter, decision_function_shape, break_ties)
#                                         if cv_acc > maxi:
#                                             maxi = cv_acc
#                                             best["C"] = C
#                                             best["gamma"] = gamma
#                                             best["coef0"] = coef0
#                                             best["tol"] = tol
#                                             best["verbose"] = verbose
#                                             best["max_iter"] = max_iter
#                                             best["decision_function_shape"] = decision_function_shape
#                                             best["break_ties"] = break_ties
#                                             best["degree"] = degree
# print("========================================================================")
# print("poly", maxi, best)
# Std scaler + uncorr: 0.8545454545454545 {'C': 1.0, 'gamma': 'scale', 'coef0': 8.0, 'tol': 0.1, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}
# Std scaler only : 0.85 {'C': 1.0, 'gamma': 'auto', 'coef0': 8.0, 'tol': 1.0, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 16}

# Robu + uncorr: 0.85 {'C': 1.0, 'gamma': 'scale', 'coef0': 10.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}
# Robu : 0.8409090909090908 {'C': 1.0, 'gamma': 'scale', 'coef0': 5.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}

# MinMax + uncorr: 0.8363636363636363 {'C': 1.0, 'gamma': 'scale', 'coef0': 0.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}
# MinMax: 0.8409090909090908 {'C': 5.0, 'gamma': 'auto', 'coef0': 8.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}

cv_acc = KF_validation_Q3(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'poly', degree=16, C=1.0, gamma='auto', coef0=8.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
my_poly_svm = SVC(kernel='poly', degree=16, C=1.0, gamma='auto', coef0=8.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
print("Q3 :", cv_acc)

# Q4:  rbf & sigmoid
# for kernel in ['rbf', 'sigmoid']:
#     maxi = 0.0
#     best = {}
#     for C in [1.0, 5.0, 20.0, 50.0, 100.0]:
#         for gamma in ['scale', 'auto']:
#             for coef0 in [-10.0, 0.0, 5.0, 8.0, 10.0, 30.0]:
#                 for tol in [0.001, 0.01, 0.1, 1.0, 30.0]:
#                     for verbose in [False, True]:
#                         for max_iter in [-1]:
#                             for decision_function_shape in ['ovr','ovo']:
#                                 for break_ties in [False, True]:
#                                     for degree in [1, 3, 5, 9, 16]:
#                                         if not(break_ties and decision_function_shape=='ovo'):
#                                             cv_acc = KF_validation_Q3(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), kernel, degree, C, gamma, coef0, tol, verbose, max_iter, decision_function_shape, break_ties)
#                                             if cv_acc > maxi:
#                                                 maxi = cv_acc
#                                                 best["C"] = C
#                                                 best["gamma"] = gamma
#                                                 best["coef0"] = coef0
#                                                 best["tol"] = tol
#                                                 best["verbose"] = verbose
#                                                 best["max_iter"] = max_iter
#                                                 best["decision_function_shape"] = decision_function_shape
#                                                 best["break_ties"] = break_ties
#                                                 best["degree"] = degree
#     print("=================================================================================================")
#     print(kernel, maxi, best)    
       
cv_acc = KF_validation_Q3(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'rbf', degree=1, C=5.0, gamma='auto', coef0=-10.0, tol=0.01, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
my_rbf_svm = SVC(kernel='rbf', degree=1, C=5.0, gamma='auto', coef0=-10.0, tol=0.01, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
print("Q4.1 (rbf):", cv_acc)

cv_acc = KF_validation_Q3(train_df.drop("labels", axis=1).to_numpy(), train_df["labels"].to_numpy(), 'sigmoid', degree=1, C=100.0, gamma='auto', coef0=5.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
my_sigm_svm = SVC(kernel='sigmoid', degree=1, C=100.0, gamma='auto', coef0=5.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False)
print("Q4.2 (sigmoid):", cv_acc)
# Standart without removing uncorrelated
# new rbf 0.8681818181818184 {'C': 5.0, 'gamma': 'auto', 'coef0': -10.0, 'tol': 0.01, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1} // 
# new sigmoid 0.7500000000000001 {'C': 100.0, 'gamma': 'auto', 'coef0': 5.0, 'tol': 1.0, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1}

# Standart
# rbf 0.8590909090909091 {'C': 20.0, 'gamma': 'auto', 'coef0': 0.0, 'tol': 0.1, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}
# sigmoid 0.8272727272727272 {'C': 1.0, 'gamma': 'scale', 'coef0': 0.0, 'tol': 0.1, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 3}

# Robust
# rbf 0.8545454545454545 {'C': 5.0, 'gamma': 'scale', 'coef0': -5.0, 'tol': 1.0, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1}
# sigmoid 0.7863636363636364 {'C': 100.0, 'gamma': 'auto', 'coef0': 5.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1}

# MinMax
# rbf 0.85 {'C': 20.0, 'gamma': 'scale', 'coef0': -5.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1}
# sigmoid 0.8227272727272726 {'C': 100.0, 'gamma': 'auto', 'coef0': 0.0, 'tol': 0.001, 'verbose': False, 'max_iter': -1, 'decision_function_shape': 'ovr', 'break_ties': False, 'degree': 1}

# Q5:
def accuracy2(pred, true):
    true = true.to_numpy()
    well = 0
    total = 0
    for i in range(len(pred)):
        if pred[i] == true[i]: 
            well += 1
        total += 1
    return well / total

# Remove not relevant feature
# Same as train_df
# for i in [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 31, 33, 34, 35, 36, 37, 38]:
#     test_df = test_df.drop("F_{}".format(i), axis=1)

# Same as train_df
columns = test_df.columns.to_numpy()
index = test_df.index.to_numpy()
rescale = preprocessing.StandardScaler()
test_df = pd.DataFrame(rescale.fit_transform(test_df), index=index, columns=columns)
test_df.loc[test_df["labels"]>0, 'labels']=1
test_df.loc[test_df["labels"]<0, 'labels']=0

print("polyacc", accuracy2(SVC(kernel='poly', degree=16, C=1.0, gamma='auto', coef0=8.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False).fit(train_df.drop("labels", axis=1), train_df["labels"]).predict(test_df.drop("labels", axis=1)), test_df["labels"]))
print("rbfacc", accuracy2(SVC(kernel='rbf', degree=1, C=5.0, gamma='auto', coef0=-10.0, tol=0.01, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False).fit(train_df.drop("labels", axis=1), train_df["labels"]).predict(test_df.drop("labels", axis=1)), test_df["labels"]))
print("sigmoidacc", accuracy2(SVC(kernel='sigmoid', degree=1, C=100.0, gamma='auto', coef0=5.0, tol=1.0, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False).fit(train_df.drop("labels", axis=1), train_df["labels"]).predict(test_df.drop("labels", axis=1)), test_df["labels"]))

Q1:  0.7727272727272727
Q2:  0.8318181818181817
Q3 : 0.85
Q4.1 (rbf): 0.8681818181818184
Q4.2 (sigmoid): 0.7500000000000001
polyacc 0.7969543147208121
rbfacc 0.8578680203045685
sigmoidacc 0.8020304568527918


In [78]:
import numpy as np
import pandas as pd
train_df = pd.read_csv("HeartTrain.csv", index_col=0)
test_df = pd.read_csv("HeartTest.csv", index_col=0)

# Tout est faux je sais pas pourquoi

def perceptron(x, y, b, w_init, eta, epoch):
    """
    Implements a binary single-example perceptron with margin
    Inputs:    x a feature matrix containing an example on each row [pandas DataFrame of shape n x d]
               y a vector with the class (either 1 or 0) of each example  [list or numpy array of size n]
               b a margin [int]
               w_init a vector with the initial weight values (intercept in w_init[0]) [list or numpy array of size d+1]
               eta a fixed learning rate [int]
               epoch the maximal number of iterations (1 epoch = 1 iteration
                       of the "repeat" loop in the lecture slides) [int]
    Output:    A weight vector [list or numpy array of size d+1] (intercept in w[0])
    """
    k, i = 0, 0
    n = len(x)
    w = np.array(w_init)
    x0 = pd.DataFrame([1]*n, index=x.index)
    x = pd.concat([x0, x], axis=1)
#     x = pd.concat([x, y], axis=1)
    check = False
    while ((not check) and (k<epoch)):
        k+=1
        i = (k % n)

        xi = x.iloc[i]
        if np.matmul(w.T, xi) <= b:
            w = w + eta*xi
        
        checker = 0
        for j in range(n):
            xj = x.iloc[j]
            if np.matmul(w.T, xj) > b:
                checker+=1
        if (checker == n):
            check = True
    return w

def predict(w, x):
    """
    Predicts the class labels of the data matrix x
    Inputs:    x a feature matrix containing an example on each row [pandas DataFrame of shape n x d]
               w a vector with the weight values computed by the perceptron (intercept in w[0]) [list or numpy array of size d+1]
    Output:    A vector of labels (1 or 0) [list or numpy array of size n]
    """
    labels=[]
    for i in range(len(x)):
        summ = w[0]
        for j in range(len(x.iloc[i])-1):
            summ += w[j + 1] * x.iloc[i][j]
        if summ >= 0:
            labels.append(1)
        else:
            labels.append(0)
    return labels   # return [1]*len(x)

n = train_df.shape[1]
print(perceptron(train_df.drop("labels", axis=1), train_df["labels"], 3, [1]*(n), 5, 5))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1]


In [77]:
d1, d2 = train_df.drop("labels", axis=1), train_df["labels"]
# d2 = pd.DataFrame([1]*len(train_df), index=train_df.index)
# print(d2)

# print(pd.concat([d2, d1], axis=1))
d2

33     1
395    0
191    1
244    1
44     1
      ..
5      1
10     1
36     1
319    0
266    1
Name: labels, Length: 220, dtype: int64